In [1]:
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
PATH_TO_ORIGINAL_DATA = '~/SecretGarden/DeepLearning/RecSy/trivagodataset/'

In [3]:
data = pd.read_csv(PATH_TO_ORIGINAL_DATA + 'train_drop_duplicates_and_after_last_clickout_items.csv', sep=',')
data.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037542,9,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


In [4]:
data=data[data["reference"].str.isnumeric()]
data.head()

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
2,00RL8Z82B2Z1,aff3928535f48,1541037532,4,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
3,00RL8Z82B2Z1,aff3928535f48,1541037532,5,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
4,00RL8Z82B2Z1,aff3928535f48,1541037542,9,interaction item image,109038,AU,"Sydney, Australia",mobile,NaN,NaN,NaN
5,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...


In [ ]:
session_lengths = data.groupby('session_id').size()
data = data[np.in1d(data.session_id, session_lengths[session_lengths>1].index)]
# item_supports = data.groupby('reference').size()
# train_item_based = data[np.in1d(data.reference, item_supports[item_supports>=5].index)]
# session_lengths = data.groupby('SessionId').size()
# data = data[np.in1d(data.SessionId, session_lengths[session_lengths>=2].index)]

In [65]:
def create_seq_db_filter_top_k(file):

    groups = file.groupby('session_id')

    # convert item ids to string, then aggregate them to lists
    aggregated = groups['reference'].agg({'sequence': lambda x: list(map(str, x))})
    init_ts = groups['timestamp'].min()
    users = groups['user_id'].min()  # it's just fast, min doesn't actually make sense

    result = aggregated.join(init_ts).join(users)
    result.reset_index(inplace=True)
    return result

In [66]:
from collections import Counter
data_util=create_seq_db_filter_top_k(data)
data_util.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  


,session_id,sequence,timestamp,user_id
0,000138ab4f789,"[2423456, 2423456, 152444]",1541446487,WDBP226XSN9E
1,0006855fc2acc,"[1255812, 1472957, 1136271, 1952929, 899487, 1...",1541485045,82L82SS4MU7W
2,0006c25fcfd9e,"[980705, 980705]",1541478280,OD8S64CCTOJV
3,000740962ae8f,"[5836036, 10039738]",1541425957,IDXWAQD6GIX3
4,0008e0d108065,"[1840085, 18370, 1840085, 1840085]",1541370339,4UV64DOIJ0NA


In [67]:
from collections import Counter
def show_stats(data_util):
    cnt = Counter()
    data_util.sequence.map(cnt.update)
    sequence_length = data_util.sequence.map(len).values
    n_sessions_per_user = data_util.groupby('user_id').size()

    print('Number of items: {}'.format(len(cnt)))
    print('Number of users: {}'.format(data_util.user_id.nunique()))
    print('Number of sessions: {}'.format(len(data_util)) )

In [68]:
show_stats(data_util)

Number of items: 14816
Number of users: 35483
Number of sessions: 38592


In [81]:
def session_split(data,
                       user_key='user_id',
                       session_key='session_id',
                       time_key='timestamp'):

    tmax = data.timestamp.max()
    session_max_times = data.groupby(session_key).timestamp.max()
    session_split = session_max_times[session_max_times < tmax-86400].index
    
    L1 = data[data.session_id.isin(session_split.values)].copy()
    L2 = data[~data.session_id.isin(session_split.values)].copy()
    return L1, L2

In [155]:
train_data,test_data=session_split(data_util)

In [156]:
show_stats(train_data);show_stats(test_data)

Number of items: 14649
Number of users: 29385
Number of sessions: 31795
Number of items: 9183
Number of users: 6577
Number of sessions: 6797


In [157]:
train_tr,valid_data=session_split(train_data)

In [158]:
show_stats(train_tr);show_stats(valid_data)

Number of items: 14260
Number of users: 22910
Number of sessions: 24659
Number of items: 9438
Number of users: 6901
Number of sessions: 7136


In [159]:
def dataset_to_format(dataset):
    lst_col = 'sequence'
    df = dataset.reset_index()
    unstacked = pd.DataFrame({
        col: np.repeat(df[col].values, df[lst_col].str.len()) for col in df.columns.drop(lst_col)}
    ).assign(**{lst_col: np.concatenate(df[lst_col].values)})[df.columns]
    # ensure that events in the session have increasing timestamps
    unstacked['timestamp'] = unstacked['timestamp'] + unstacked.groupby('user_id').cumcount()
    unstacked.rename(columns={'sequence': 'item_id'}, inplace=True)
    return unstacked

In [160]:
train_tr=dataset_to_format(train_tr)
valid_data=dataset_to_format(valid_data)
test_data=dataset_to_format(test_data)

In [162]:
train_tr.drop(['index'],axis=1).to_csv('rsc19_train.txt', sep='\t', index=False)
valid_data.drop(['index'],axis=1).to_csv('rsc19_valid.txt', sep='\t', index=False)
test_data.drop(['index'],axis=1).to_csv('rsc19_test.txt', sep='\t', index=False)